# 💻 Notebook: Entrenamiento y Evaluación del Modelo SARIMAX Vectorial

### **Paso 1: Configuración, Carga de Datos y Parámetros**

Cargamos las librerías necesarias para el modelado, los datos estacionarios que creamos y, crucialmente, los parámetros de reversión.

In [1]:
import pandas as pd
import numpy as np
import json
# ¡SOLUCIÓN! Usamos VARMAX para modelos vectoriales
from statsmodels.tsa.statespace.varmax import VARMAX 
import itertools
import warnings
import joblib # Necesario para guardar el modelo

# Ignorar advertencias de convergencia durante el Grid Search
warnings.filterwarnings("ignore") 

# --- A. CARGA DE DATOS ESTACIONARIOS ---
# Asegúrate de que las rutas sean correctas
try:
    df_stationary = pd.read_csv('datos_temperaturas_estacionarios.csv', index_col='Fecha', parse_dates=True)
except FileNotFoundError:
    df_stationary = pd.read_csv('dataset/datos_temperaturas_estacionarios.csv', index_col='Fecha', parse_dates=True)

# --- B. CARGA DE PARÁMETROS DE REVERSIÓN ---
try:
    with open('parametros_reversion.json', 'r') as f:
        params_reversion = json.load(f)
except FileNotFoundError:
    with open('dataset/parametros_reversion.json', 'r') as f:
        params_reversion = json.load(f)
    
if df_stationary is not None and params_reversion is not None:
    print(f"✅ Datos estacionarios cargados (N={len(df_stationary)}).")
    print(f"✅ Parámetros de reversión cargados. Última fecha histórica: {params_reversion['ultima_fecha_historica']}.")

✅ Datos estacionarios cargados (N=670).
✅ Parámetros de reversión cargados. Última fecha histórica: 2025-11-01.


### **Paso 2: Búsqueda por Rejilla (Grid Search) de Órdenes Óptimos**

Encontramos la combinación de órdenes $(p, q)$ (no estacional) y $(P, Q)$ (estacional) que minimice el Criterio de Información de Akaike (AIC), que equilibra el ajuste del modelo y su complejidad.

**Nota:** Como es un modelo Vectorial (SARIMAX para series múltiples), los órdenes se aplican al sistema en su conjunto.

In [2]:
# --- 2.1. DEFINICIÓN DE RANGOS DE BÚSQUEDA ---
# VARMAX: Solo requiere órdenes AR (p) y MA (q) para los datos ya estacionarios.
p = q = range(0, 3) # p, q de 0 a 2
P = Q = range(0, 2) # P, Q estacionales de 0 a 1
S = 365 # Estacionalidad anual

# Creamos todos los posibles pares (p, q) y (P, Q)
pq = list(itertools.product(p, q))
PQ = list(itertools.product(P, Q))

# Generamos la lista de tuplas de órdenes VARMAX
VARMAX_orders = list(itertools.product(pq, PQ))

# --- 2.2. FUNCIÓN DE BÚSQUEDA (CORREGIDA) ---
def grid_search_varmax(data, varmax_orders):
    """Realiza la búsqueda por rejilla para encontrar el modelo con menor AIC."""
    best_aic = np.inf
    best_order = None
    best_seasonal_order = None

    for order in varmax_orders:
        try:
            order_ns = order[0]  # (p, q)
            order_s = order[1] + (S,) # (P, Q, S)
            
            # ¡CLAVE! Usamos VARMAX aquí
            model = VARMAX(
                data,
                order=order_ns,
                seasonal_order=order_s,
                # En VARMAX no se usan enforce_stationarity ni enforce_invertibility
            ).fit(maxiter=50, disp=False) 

            if model.aic < best_aic:
                best_aic = model.aic
                best_order = order_ns
                best_seasonal_order = order_s

        except Exception as e:
            # print(f"Error con órdenes {order}: {e}")
            continue

    return best_order, best_seasonal_order, best_aic

# Ejecutar el Grid Search
print("\nIniciando Grid Search para órdenes VARMAX (Minimizar AIC)...")
optimal_order, optimal_seasonal_order, optimal_aic = grid_search_varmax(
    df_stationary,
    VARMAX_orders
)

print("\n=======================================================")
print("🏆 RESULTADOS DEL GRID SEARCH")
print(f"Órdenes No Estacionales (p, q): {optimal_order}") 
print(f"Órdenes Estacionales (P, Q, S): {optimal_seasonal_order}")
print(f"Mínimo AIC Obtenido: {optimal_aic:.2f}")
print("=======================================================")


Iniciando Grid Search para órdenes VARMAX (Minimizar AIC)...

🏆 RESULTADOS DEL GRID SEARCH
Órdenes No Estacionales (p, q): (2, 1)
Órdenes Estacionales (P, Q, S): (0, 0, 365)
Mínimo AIC Obtenido: 32686.95


### **Paso 3: Entrenamiento Final y Resumen**

Entrenamos el modelo con los órdenes óptimos encontrados para obtener el resumen estadístico.

In [3]:
# --- 3.1. ENTRENAMIENTO FINAL (CORREGIDO) ---
# ¡CLAVE! Usamos VARMAX con los órdenes óptimos.
final_model = VARMAX(
    df_stationary,
    order=optimal_order,
    seasonal_order=optimal_seasonal_order,
).fit(disp=False, maxiter=100) # Más iteraciones para el modelo final

# --- 3.2. REPORTE DEL MODELO ---
print("\n\n*******************************************************")
print("               RESUMEN DEL MODELO VARMAX")
print("*******************************************************")
print(final_model.summary())
print("*******************************************************\n\n")



*******************************************************
               RESUMEN DEL MODELO VARMAX
*******************************************************
                                     Statespace Model Results                                     
Dep. Variable:     ['Diff_BoxCox_Max', 'Diff_BoxCox_Min']   No. Observations:                  670
Model:                                         VARMA(2,1)   Log Likelihood              -16325.852
                                              + intercept   AIC                          32685.704
Date:                                    Mon, 15 Dec 2025   BIC                          32762.328
Time:                                            08:08:15   HQIC                         32715.383
Sample:                                        01-02-2024                                         
                                             - 11-01-2025                                         
Covariance Type:                                     